In [8]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Lg-Stock-Price-Prediction'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [9]:
data = pd.read_csv('./data/lg.csv')

In [10]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-08-23,52700.0,52800.0,51400.0,51500.0,49258.472656,737608.0
1,2016-08-24,51600.0,51700.0,51000.0,51100.0,48875.886719,653161.0
2,2016-08-25,51400.0,52300.0,51400.0,52100.0,49832.359375,579633.0
3,2016-08-26,52100.0,52200.0,51500.0,52100.0,49832.359375,354347.0
4,2016-08-29,52100.0,52200.0,51400.0,51500.0,49258.472656,353607.0
...,...,...,...,...,...,...,...
1223,2021-08-17,150500.0,153500.0,150000.0,151000.0,151000.000000,779899.0
1224,2021-08-18,150500.0,154000.0,150000.0,153500.0,153500.000000,652995.0
1225,2021-08-19,151500.0,153000.0,148000.0,148000.0,148000.000000,737444.0
1226,2021-08-20,148500.0,150500.0,146000.0,146500.0,146500.000000,802093.0


In [11]:
data = torch.from_numpy(np.array(data['Low'].tolist())).view(1,-1)

In [13]:
data_input = data[:1,:-1].float().to(device)
data_target = data[:1,1:].float().to(device)

In [14]:
data_input

tensor([[ 51400.,  51000.,  51400.,  ..., 150000., 148000., 146000.]],
       device='cuda:0')

In [15]:
data_target

tensor([[ 51000.,  51400.,  51500.,  ..., 148000., 146000., 138000.]],
       device='cuda:0')

In [ ]:
class Model(Module):
    def __init__(self,hidden=128):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        